In [1]:
import cv2
import torch

In [2]:
def gstreamer_pipeline(
    sensor_id=0,
    capture_width=1280,
    capture_height=720,
    display_width=1280,
    display_height=720,
    framerate=60,
    flip_method=0,
):
    return (
        "nvarguscamerasrc sensor-id=%d ! "
        "video/x-raw(memory:NVMM), width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            sensor_id,
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

In [ ]:
def detect(model):
    COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255)]  # 다양한 색상
    window_title = "YOLOv5 detection"
    inference_times_mean = []

    # To flip the image, modify the flip_method parameter (0 and 2 are the most common)
    print(gstreamer_pipeline(flip_method=0))
    video_capture = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
    if video_capture.isOpened():
        try:
            window_handle = cv2.namedWindow(window_title, cv2.WINDOW_AUTOSIZE)
            while True:
                ret_val, frame = video_capture.read()
                if not ret_val:
                    break
                start_event = torch.cuda.Event(enable_timing=True)
                end_event = torch.cuda.Event(enable_timing=True)
                
                with torch.no_grad():
                    start_event.record()
                    results = model(frame, size=1280, augment=True)
                    end_event.record()
                    
                torch.cuda.synchronize()
                elapsed_time_ms = start_event.elapsed_time(end_event)
                
                for *xyxy, conf, cls in results.xyxy[0].tolist():
                    x1, y1, x2, y2 = map(int, xyxy)
                    label = f"{results.names[int(cls)]} {conf:.2f}"
                    color = COLORS[int(cls) % len(COLORS)]
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                    label_x1, label_y1 = x1, y1 - label_size[1] - 10
                    label_x2, label_y2 = x1 + label_size[0] + 10, y1
                    cv2.rectangle(frame, (label_x1, label_y1), (label_x2, label_y2), color, -1)
                    cv2.putText(frame, label, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                cv2.imshow(window_title, frame)
                print('inference time : {}ms'.format(elapsed_time_ms))
                inference_times_mean.append(elapsed_time_ms)
                keyCode = cv2.waitKey(10) & 0xFF
                # Stop the program on the ESC key or 'q'
                if keyCode == 27 or keyCode == ord('q'):
                    break
        finally:
            video_capture.release()
            cv2.destroyAllWindows()
            print("\nMean inference time : {}ms".format(sum(inference_times_mean[5:]) / len(inference_times_mean[5:])))
    else:
        print("Error: Unable to open camera")

In [20]:
import os

def prune(model, amount=0.3):
    import torch.nn.utils.prune as prune
    print('Pruning model... ', end='')
    for name, m in model.named_modules():
        if isinstance(m, torch.nn.Conv2d):
            prune.l1_unstructured(m, name='weight', amount=amount)  # prune
            prune.remove(m, 'weight')  # make permanent
    print(' %.3g global sparsity' % sparsity(model))
    return model
            
def sparsity(model):
    # Return global model sparsity
    a, b = 0, 0
    for p in model.parameters():
        a += p.numel()
        b += (p == 0).sum()
    return b / a

def quantize(model):
    model = model.half()  # 모델의 가중치를 FP16으로 변환
    for layer in model.modules():
        if isinstance(layer, torch.nn.BatchNorm2d):
            layer.float()  # BatchNorm 레이어는 FP32로 유지
    return model

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [5]:
def real_prune(model, pruning_ratio=0.5):
    import torch_pruning as tp
    #print(model.model)
    for p in model.parameters():
        p.requires_grad_(True)

    example_inputs = torch.randn(1, 3, 640, 640).to(device)
    imp = tp.importance.MagnitudeImportance(p=2) # L2 norm pruning

    ignored_layers = []
    from models.yolo import Detect
    for m in model.model.modules():
        if isinstance(m, Detect):
            ignored_layers.append(m)
    #print(ignored_layers)

    iterative_steps = 1 # progressive pruning
    pruner = tp.pruner.MagnitudePruner(
        model.model,
        example_inputs,
        importance=imp,
        iterative_steps=iterative_steps,
        pruning_ratio=pruning_ratio, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
        ignored_layers=ignored_layers,
    )

    
    base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
    for g in pruner.step(interactive=True):
        print(g)
        g.prune()

    pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(model, example_inputs)
    #print(model)
    print("Before Pruning: MACs=%f G, #Params=%f G"%(base_macs/1e9, base_nparams))
    print("After Pruning: MACs=%f G, #Params=%f G"%(pruned_macs/1e9, pruned_nparams))
    return model

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda:0


In [7]:
%cd ~/Documents/Glasses_Detection_with_YOLOv5/

/home/jetson/Documents/Glasses_Detection_with_YOLOv5


In [24]:
model = torch.hub.load('yolov5', 'custom', 'good_weights/yolov5n_wandb12.pt', source='local', force_reload=True, device=device)
#model = torch.hub.load('ultralytics/yolov5', "custom", "models/best.pt", force_reload=True)

YOLOv5 🚀 af7761d torch 1.8.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)

Fusing layers... 
Model Summary: 213 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


In [25]:
model = real_prune(model, pruning_ratio=0.1)
model = quantize(model)


--------------------------------
          Pruning Group
--------------------------------
[0] prune_out_channels on model.model.23.cv3.conv (Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))) => prune_out_channels on model.model.23.cv3.conv (Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))), len(idxs)=26
[1] prune_out_channels on model.model.23.cv3.conv (Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))) => prune_out_channels on _ElementWiseOp_21(SiluBackward), len(idxs)=26
[2] prune_out_channels on _ElementWiseOp_21(SiluBackward) => prune_out_channels on _ElementWiseOp_19(CudnnConvolutionBackward), len(idxs)=26
[3] prune_out_channels on _ElementWiseOp_19(CudnnConvolutionBackward) => prune_in_channels on model.model.24.m.2 (Conv2d(256, 21, kernel_size=(1, 1), stride=(1, 1))), len(idxs)=26
[4] prune_in_channels on model.model.24.m.2 (Conv2d(256, 21, kernel_size=(1, 1), stride=(1, 1))) => prune_out_channels on _Reshape_20(), len(idxs)=26
--------------------------------


-------

In [26]:
detect(model)

nvarguscamerasrc sensor-id=0 ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, framerate=(fraction)60/1 ! nvvidconv flip-method=0 ! video/x-raw, width=(int)1280, height=(int)720, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink
inference time : 1799.6085205078125ms
inference time : 12785.095703125ms
inference time : 558.0092163085938ms
inference time : 372.79156494140625ms
inference time : 376.9693603515625ms
inference time : 369.95391845703125ms
inference time : 372.9580078125ms
inference time : 384.30865478515625ms
inference time : 375.5967102050781ms
inference time : 375.6368713378906ms
inference time : 374.94134521484375ms
inference time : 375.2831726074219ms
inference time : 375.3145446777344ms
inference time : 377.639892578125ms
inference time : 376.5321960449219ms
inference time : 374.5437927246094ms
inference time : 375.92047119140625ms
inference time : 379.94183349609375ms
inference time : 375.8141174316406ms
inference time : 373.11587